# Chatbot : webscraping & human interaction

As someone in Data Science with experience in human-in-the-loop and real-time human measures, designing and programming a human interaction chatbot sounded interesting.  Without following a known chatbot structure, I tried to be creative and come up with my own logical structure.  

I looked at a few medium blogs, and I tried to first make a "Self-learning Retrieval Based chatbot" over a Rule-based chatbot or Self-learning Generative chatbot.

Check out github for the supporting python subfunctions : https://github.com/j622amilah/Chatbot!

In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
import numpy as np

import nltk

# Personal python functions
import sys
sys.path.insert(1, 'C:\\Users\\jamilah\\Documents\\Subfunctions_python')

from findall import *
from dictionary.sort_dict_by_value import *
from string_text_processing.text_url_2_senANDwordtokens import *
from string_text_processing.preprocessing import *
from string_text_processing.get_word_count_uniquewords import *
from string_text_processing.remove_chars_from_wordtoken import *
from string_text_processing.get_cossine_similarity import *

# Get Knowlege base for chatbot by Webscraping

In [189]:
inputurl = "https://en.wikipedia.org/wiki/Chatbot"
name_txtfile = 'chatbot_wikipedia'
sen, word_tok = text_url_2_senANDwordtokens(inputurl, name_txtfile)

There are 209 sentences


In [191]:
inputurl = "https://en.wikipedia.org/wiki/Q%26A_software"
name_txtfile = 'chatbot_wikipedia'
sen, word_tok = text_url_2_senANDwordtokens(inputurl, name_txtfile)

There are 82 sentences


In [182]:
inputurl = "https://preply.com/en/blog/22-useful-english-greetings-for-every-day/"
name_txtfile = 'how2sayHello'
sen, word_tok = text_url_2_senANDwordtokens(inputurl, name_txtfile)

There are 1493 sentences


In [10]:
inputurl = "https://regardsurlefrancais.com/2017/09/19/les-salutations-en-francais/"
name_txtfile = 'commentDitBonjour'
sen, word_tok = text_url_2_senANDwordtokens(inputurl, name_txtfile)

There are 1591 sentences


In [192]:
sen[0]

['b',
 'QampA',
 'software',
 'is',
 'software>online',
 'that',
 'attempts',
 'to',
 'answer',
 'questions',
 'asked',
 'by',
 'users',
 'QampA',
 'stands',
 'for',
 'question',
 'and']

# Retrieval Based chatbot

# Logic and Parser Rules

Give the chatbot text, and it will behave based on the text.
If you want it to say 'greetings', give it text on 'greetings' and 'manners'.

Need to keep the texts separate maybe, based on text A it would say AAA, 
and based on text B it would say BBB.  Then decide if AAA or BBB is better, based on similarity maybe

In [193]:
# Get the theme of the knowledge base
word_tokens2 = preprocessing(word_tok)

list_to_remove = ['b', "their", "based", "which", 'would', 'https']
wc, keywords, mat_sort = get_word_count_uniquewords(word_tokens2, list_to_remove)

print("Hello, I am a chatbot and I can talk about the following topics : ", keywords[0:10])
#print("Top 10 wc : ", wc[0:10])

There are 921 word tokens, but 337 words are unique.
[['47' 'qampa']
 ['44' 'answer']
 ['31' 'question']
 ['18' 'commun']
 ['15' 'onlin']
 ['15' 'peopl']
 ['14' 'softwar']
 ['14' 'motiv']
 ['14' 'servic']
 ['12' 'contribut']
 ['11' 'knowledg']
 ['11' 'social']
 ['10' 'yahoo']
 ['8' 'provid']
 ['8' 'inform']
 ['7' 'behavior']
 ['7' 'featur']
 ['7' 'launch']
 ['6' 'qualiti']
 ['6' 'expert']
 ['6' 'software']
 ['6' 'googl']
 ['6' 'connect']
 ['6' 'sites']
 ['5' 'questions']
 ['5' 'graph']
 ['5' 'stack']
 ['5' 'number']
 ['5' 'allow']
 ['4' 'contributor']
 ['4' 'comparison']
 ['4' 'factor']
 ['4' 'specialist']
 ['4' 'share']
 ['4' 'support']
 ['4' 'increas']
 ['4' 'exchange']
 ['4' 'knowledge']
 ['4' 'survey']
 ['4' 'system']
 ['4' 'member']
 ['4' 'differ']
 ['4' 'websit']
 ['4' 'point']
 ['4' 'expect']
 ['3' 'exampl']
 ['3' 'exchang']
 ['3' 'signific']
 ['3' 'relat']
 ['3' 'reward']
 ['3' 'three']
 ['3' 'involv']
 ['3' 'network']
 ['3' 'potenti']
 ['3' 'similar']
 ['3' 'includ']
 ['3' 'pa

In [194]:
# Get user input about the topics it can talk about
inp = input("Type a question to the chatbot: ")
 
# prints inp
print(inp)
inp = 'when were chatbots first made'
inp = 'when did chatbots become popular'
inp = 'when was qampa first made'

Type a question to the chatbot: when was qampa first made
when was qampa first made


In [195]:
# Determine which sentences are relate to the main themes and {who, what, when, where, why}

# who = name, he, she, Mr, Mrs, Ms
# what = keywords
# when = number!  ---> search for months, dates, time, year, when was
# where = location
# why = because, due to the fact

# verbs ?? = action

# word token input
inp_wt0 = inp.split()
print('inp_wt0: ', inp_wt0)

# Stem the input text
ps = PorterStemmer()
inp_wt = []
for w in inp_wt0:
    inp_wt.append(ps.stem(w))
print('inp_wt: ', inp_wt)

# Simple framework for setting up a Q&A transformer model: Retrival of key sentences in the knowledge data source
q_type = []
for i in inp_wt:
    if i in ['who', 'name of']:
        q_type.append('who')
        # Need a function that detects names
        list_to_search = ['he', 'she', 'they', 'Mr', 'Mrs', 'Ms']
        # Search knowledge for sentences that contain names, pronouns (he, she, they, Mr, Mrs, Ms)
    elif i in ['what']:
        q_type.append('what')
        # Search knowledge for sentences that contain knowledge keywords similar to user keywords
    elif i in ['when']:
        q_type.append('when')  # ONLY DOING 'when' for the moment
    elif i in ['where']:
        q_type.append('where')
    elif i in ['why']:
        q_type.append('why')
q_type        

inp_wt0:  ['when', 'was', 'qampa', 'first', 'made']
inp_wt:  ['when', 'wa', 'qampa', 'first', 'made']


['when']

In [196]:
# Based on the Question Type: find the relavent sentences in the Knowledge Source that could Answer the Question Type
for i in q_type:
    
    if i == 'when':
        # -----------------------------------------
        # What to do if "when" is in the input:
        # -----------------------------------------
        out = []
        for i in word_tok:
            # Search to see if there are '.' at the end of the word
            i = remove_chars_from_wordtoken(i, '.', '')
            i = remove_chars_from_wordtoken(i, '%', '')
            i = remove_chars_from_wordtoken(i, '$', '')

            # First determine if a number is a ratio
            i = remove_chars_from_wordtoken(i, '/', ' divided by ')

            # Second determine if a number is present
            if i.upper() == i.lower() and i != '':
                # is a number
                out.append(int(i))

        imp_nums = np.unique(out)
        imp_nums_str = [str(i) for i in imp_nums]
        
        # Search knowledge for sentences that contain numbers and text referring to time 
        # -----------------------------------------
        # 1) Get the sentences that contain numbers
        sen_with_nums = []
        for ind, i in enumerate(sen):
            # Returns the unique values in 1st array NOT in 2nd array
            vals_from_long_not_in_short = np.setdiff1d(imp_nums_str, i)
            out = np.setdiff1d(imp_nums_str, vals_from_long_not_in_short)
            if any(out):
                sen_with_nums.append(ind)
        # -----------------------------------------
        
        # -----------------------------------------
        # 2) Get the sentences 
        list_to_search = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December', 'year', 'month', 'week', 'day', 'hour', 'minute', 'second', 'ms', 'millisecond']
        sen_with_timetext = []
        for ind, i in enumerate(sen):
            # Returns the unique values in 1st array NOT in 2nd array
            vals_from_long_not_in_short = np.setdiff1d(list_to_search, i)
            out = np.setdiff1d(list_to_search, vals_from_long_not_in_short)
            if any(out):
                sen_with_timetext.append(ind)
        
        # -----------------------------------------
        
print('sen_with_nums: ', sen_with_nums)
print('sen_with_timetext: ', sen_with_timetext)

# Combine 'relevant sentence' lists
sen_with_nums += sen_with_timetext
relv_sen_unique = np.unique(sen_with_nums)
print('relv_sen_unique: ', relv_sen_unique)

sen_with_nums:  [9, 11, 14, 16, 17, 21, 23, 24, 41, 63, 65, 69]
sen_with_timetext:  [11, 14, 16, 17, 18, 21, 23, 36]
relv_sen_unique:  [ 9 11 14 16 17 18 21 23 24 36 41 63 65 69]


In [197]:
# Search for common words in the input with each of 'relevant sentences'
cos_sim_all = get_cossine_similarity(inp_wt, relv_sen_unique)
cos_sim_all = make_a_properlist(cos_sim_all)

# 1) Get a Percentage of how many words of 'input' are used in sen
dict_val = {}
for ind, rels in enumerate(relv_sen_unique):
    tot = np.zeros((len(inp_wt),1))
    for i in range(len(inp_wt)):
        for j in range(len(sen[rels])):
            if inp_wt[i] == sen[rels][j]:
                tot[i] = 1
                break

    per = sum(np.ravel(tot))/len(inp_wt) * 100
    
    dict_val[rels] = [per, cos_sim_all[ind], len(sen[rels])]  # Percentage and cos-sine similarity

# Sort by max
dict_val_sorted = sort_dict_by_value(dict_val, reverse = True)
dict_val_sorted

C:\Users\jamilah\AppData\Local\Temp/ipykernel_9672/2203938722.py:41: RuntimeWarning: invalid value encountered in true_divide
  cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))


{21: [20.0, 0.4472135954999579, 18],
 9: [20.0, 0.4472135954999579, 16],
 11: [0.0, nan, 22],
 14: [0.0, nan, 18],
 16: [0.0, nan, 10],
 17: [0.0, nan, 26],
 18: [0.0, nan, 6],
 23: [0.0, nan, 18],
 24: [0.0, nan, 24],
 36: [0.0, nan, 20],
 41: [0.0, nan, 33],
 63: [0.0, nan, 43],
 65: [0.0, nan, 16],
 69: [0.0, nan, 24]}

In [198]:
# So the Longest sentence WITH the Highest similarity appears to be the best response!!
chatbot_output = sen[list(dict_val_sorted.keys())[0]]


vals_from_long_not_in_short = np.setdiff1d(imp_nums_str, chatbot_output)
out = np.setdiff1d(imp_nums_str, vals_from_long_not_in_short)
if any(out):
    print('number_val', out)

print('chatbot_output : ', chatbot_output)

number_val ['2009' '21']
chatbot_output :  ['b', 'was', 'founded', 'in', 'June', '2009', 'while', 'the', 'website', 'was', 'made', 'available', 'to', 'the', 'public', 'on', 'June', '21']


I asked some simple questions, so I think I got Luckly!

I think it could be improved with a Q&A transformer step with the 'relavant sentences'!  And, I was a little lazy about separating the sentences perfectly using the webscraped words.  If the sentences were perfectly separated, it would be interesting to see how it performs!